In [22]:
import random
import re
import pandas as pd
import os

from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

def lm_emulator(prompt):
    ### DIES IST KEINE PROMT SONDERN EIN TEST
    print(prompt)
    test_text = f"""
        Ich werde die bereitgestellten Daten analysieren und eine Einschätzung zur Wirkung der Impfkampagne auf die Infektionsrate in NRW im Januar 2022 vornehmen.

        1. Analyse der aktuellen Situation:
        Infektionsrate: 0.5
        Fallzahlen: 1000
        Demografische Impftrends: Frauen werden häufiger geimpft als Männer. Menschen mit Hypertonie haben eine höhere Impfquote, besonders über 60-Jährige.
        Regionale Impfquote: NRW hat die höchste Impfquote unter den Bundesländern.
        Bevölkerungsstruktur: NRW hat eine hohe Bevölkerungsdichte, was die Virusübertragung begünstigt.
        Impfquote nach Alter: Ältere Menschen, besonders über 60, werden häufiger geimpft, was den Schutz der vulnerablen Gruppen verbessert.
        2. Wissenschaftliche Einschätzung der Impfkampagnenwirkung:
        Impfkampagnen führen kurzfristig zu einer Reduktion der Infektionsrate, da mehr Menschen immunisiert werden, was die Übertragung reduziert.
        Herdenimmunitätseffekt: Da die Impfquote unter älteren Menschen hoch ist, wird erwartet, dass schwere Krankheitsverläufe reduziert werden.
        Geschlechtsspezifischer Einfluss: Männer haben eine niedrigere Impfquote, was potenziell zu einer höheren Infektionsrate in dieser Gruppe führen könnte.
        Einfluss der Privatversicherung: Da Männer tendenziell häufiger privat versichert sind, könnte eine unterschiedliche Impfpriorisierung eine Rolle spielen.
        3. Prognose für die Infektionsrate in NRW nach der Impfkampagne:
        Die Impfkampagne wird voraussichtlich die Infektionsrate senken, jedoch nicht sofort drastisch, da Impfungen einige Zeit benötigen, um eine Immunreaktion zu erzeugen. Basierend auf bisherigen Impfkampagnen und der aktuellen Ausgangslage, könnte die Infektionsrate um 10–20% gesenkt werden.

        Geschätzte Infektionsrate nach der Impfkampagne:
        Infektionsrate: [[[{round(random.uniform(0.1, 2), 2)}]]]
    """
    return test_text

def lm(prompt):
    openai_api_key = os.getenv("OPENAI_API_KEY")
    # Set your OpenAI API key
    client = OpenAI(api_key=openai_api_key)

    completion = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": prompt
            }
        ]
    )

    return completion.choices[0].message.content

In [23]:
# output = lm("How are you?")
# print(output)

In [24]:
from sqlalchemy import create_engine, Column, String, Float, Integer
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

Base = declarative_base()

class CacheEntry(Base):
    __tablename__ = 'cache_entries901'
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    Region = Column(String)
    Year = Column(Integer)
    Month = Column(Integer)
    Percentage = Column(Float)
    InfectionRate = Column(Float)
    PreviousInfectionRate = Column(Float)
    Risikogruppe = Column(String)
    Altersgruppe = Column(String)
    Prompt = Column(String)
    Output = Column(String)

# Initialize the database
engine = create_engine('sqlite:///cache.db')
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)

class Cache:
    def __init__(self):
        """Initialize the cache with SQLAlchemy session."""
        self.session = Session()

    def add_to_cache(self, region, year, month, percentage, infection_rate, prompt, output, risikogruppe, altersgruppe, previous_infection_rate=None):
        """Add or update a row in the cache."""

            # Add new entry
        entry = CacheEntry(
            Region=region,
            Year=int(year) if year else None,
            Month=int(month) if month else None,
            Percentage=float(percentage) if percentage else None,
            InfectionRate=float(infection_rate) if infection_rate else None,
            PreviousInfectionRate=float(previous_infection_rate) if previous_infection_rate else None,
            Risikogruppe=risikogruppe,
            Altersgruppe=altersgruppe,
            Prompt=prompt,
            Output=output
        )
        self.session.add(entry)
        self.session.commit()

    def get_from_cache(self, region, year, month, risikogruppe, altersgruppe):
        """Retrieve a value from the cache."""
        entry = self.session.query(CacheEntry).filter_by(Region=region, Year=year, Month=month, Risikogruppe=risikogruppe, Altersgruppe=altersgruppe).first()
        if entry:
            return {
                'Region': entry.Region,
                'Year': entry.Year,
                'Month': entry.Month,
                'Percentage': entry.Percentage,
                'InfectionRate': entry.InfectionRate,
                'PreviousInfectionRate': entry.PreviousInfectionRate,
                'Risikogruppe': entry.Risikogruppe,
                'Altersgruppe': entry.Altersgruppe,
                'Prompt': entry.Prompt,
                'Output': entry.Output
            }
        return None


C:\Users\Amaan\AppData\Local\Temp\ipykernel_60768\1598011961.py:5: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [25]:
def extract_between_brackets(text):
    """
    Extracts and returns the content inside square brackets from a string.
    If multiple brackets exist, it returns a list of all matches.

    :param text: The input string.
    :return: A string if one match, a list if multiple matches, or None if no match.
    """
    matches = re.findall(r'\[\[\[(.*?)\]\]\]', text)  # Extract content inside square brackets
    if not matches:
        return None  # Return None if no matches found
    return matches if len(matches) > 1 else matches[0]  # Return list if multiple, string if one


In [26]:
def parse_lm_output(output):
    """
    Parses the output from the language model and extracts the predicted values.

    :param output: The output text from the language model.
    :return: The predicted value(s) as a string or list of strings.
    """

    rate_str = extract_between_brackets(output)
    if rate_str is not None:
        if type(rate_str) != list:
            try:
                return float(rate_str)
            except ValueError:
                return None
        else:
            try:
                return float(rate_str[0])
            except ValueError:
                return None
    return

In [27]:
# Inputs für ein paar der Variablen
# Altergrsuppe: über 60 jährige, unter 60 jährige,
# Risikogruppe: hoher (Risiko), niedriger (Risiko)

def generate_prompt(bundesland, jahr, monat, fallzahlen, infektionsrate, risikogruppe, altersgruppe):
    # Hauptanweisung / Zieldefinition (klar formuliert)
    general_prompt = (
        f"Du bist ein erfahrener Epidemiologe mit fundiertem Wissen über Demografie, Gesundheitsinfrastruktur und wissenschaftliche Studien zu Impfungen. Deine Aufgabe ist es, die Wirksamkeit einer Impfkampagne in {bundesland} für den Monat {monat} im Jahr {jahr} einzuschätzen, insbesondere hinsichtlich der Reduktion der Infektionsrate bei Personen aus der Altersgruppe {altersgruppe} und mit {risikogruppe}. Beantworte dazu bitte die folgenden Punkte Schritt für Schritt. Dein Output sollte aber nicht länger als ein Paragraph sein:\n\n"

        f"1. Analysiere die bereitgestellten Daten zur aktuellen Impfquote, Infektionsrate und weiteren relevanten Faktoren.\n"
        f"2. Berücksichtige dein Wissen über die Region (einschließlich Demografie, Gesundheitsinfrastruktur und bisherige Impfergebnisse), um abzuleiten, wie sich diese Faktoren voraussichtlich auf die Infektionsrate auswirken.\n"
        f"3. Erkläre auf Basis wissenschaftlicher Erkenntnisse kurz, wie effektiv die Impfkampagne sein könnte und wodurch sich diese Wirksamkeit begründet.\n"
        f"4. Gib abschließend eine Prognose für die zu erwartende Infektionsrate in {bundesland}, gefolgt von einer kurzen Begründung. Verwende zur Ausgabe deiner endgültigen Schätzung unbedingt das Format: Infektionsrate: [[[Wert]]] d.h. z.B. [[[0.0926]]]\n\n"
    )

    # Datenkontext
    data_knowledge = (f"Relevante Daten aus {bundesland} für den Monat {monat} im Jahr {jahr}: "
                      f"Aktuell sind {round(fallzahlen, 6)}% der Menschen in dieser Gruppe infiziert.\n")
    if infektionsrate:
        data_knowledge += (
            f"Die Infektionsrate des letzten Monats beträgt {round(infektionsrate, 4)}.\n"
        )
    else:
        data_knowledge += "Aktuell liegen keine konkreten Fallzahlen vor.\n"

    # Weiterführende Hintergrundinformationen (Exploration Knowledge)
    exploration_knowledge = (
        "Zusätzliche Erkenntnisse aus bisherigen Untersuchungen:\n"
        "- Frauen lassen sich deutlich häufiger impfen als Männer.\n"
        "- Männer sind tendenziell etwas öfter privat versichert als Frauen.\n"
        "- Menschen mit Hypertonie werden häufiger geimpft als Menschen mit anderen Vorerkrankungen.\n"
        "- Die Impfquote ist in der Altersgruppe über 60 am höchsten.\n"
        "- Bei Risikogruppen: Über-60-Jährige werden exponentiell häufiger geimpft als "
        "- Unter-60-Jährige (außer bei Asthma, wo ein linearer Anstieg vorliegt).\n"
        "- Unter Berücksichtigung der Bevölkerungsdichte ist die Impfquote in den verschiedenen Bundesländern sehr ähnlich.\n"
        "- Ebenfalls unter Berücksichtigung der Bevölkerungsdichte nimmt die Impfquote linear von 7,8 % auf 4,3 % ab. Die Bundesländer sind absteigend wie folgt gerankt: Nordrhein-Westfalen, Niedersachsen, Hessen, Baden-Württemberg, Bayern, Bremen, Sachsen-Anhalt, Schleswig-Holstein, Rheinland-Pfalz, Thüringen, Hamburg, Brandenburg, Berlin, Mecklenburg-Vorpommern, Sachsen, Saarland.\n"
    )

    return general_prompt + data_knowledge + exploration_knowledge


In [28]:
df = pd.read_csv("final_df_neu.csv", index_col=0, header=0)
df

,Region,Altersgruppe,Inzidenz,Jahr,Woche,Time,risk_groups,Fallzahl,Inzidenz_cumsum,cumsum
2629,Baden-Württemberg,0-29,0.00964,2024,1,2024-01-01,no,0.002757,0.002757,0.00964
2677,Baden-Württemberg,0-29,0.01069,2024,2,2024-01-08,no,0.003057,0.005814,0.02033
2725,Baden-Württemberg,0-29,0.02613,2024,3,2024-01-15,no,0.007473,0.013288,0.04646
2773,Baden-Württemberg,0-29,0.04003,2024,4,2024-01-22,no,0.011449,0.024736,0.08649
2821,Baden-Württemberg,0-29,0.04293,2024,5,2024-01-29,no,0.012278,0.037014,0.12942
...,...,...,...,...,...,...,...,...,...,...
2436,Thüringen,60-199,0.06115,2025,5,2025-01-27,yes,0.043661,0.301694,0.15654
2484,Thüringen,60-199,0.04948,2025,6,2025-02-03,yes,0.035329,0.337022,0.20602
2532,Thüringen,60-199,0.06434,2025,7,2025-02-10,yes,0.045939,0.382961,0.27036
2580,Thüringen,60-199,0.05558,2025,8,2025-02-17,yes,0.039684,0.422645,0.32594


In [29]:
# cumsum

In [30]:
df = df.rename(columns={
    "Jahr": "Year",
    "Woche": "CalendarWeek",
    "risk_groups" : "Risikogruppe",
    "Fallzahl" : "Percentage",
    "cumsum" : "Percentage_cumsum"
})

In [31]:
df['Date'] = pd.to_datetime(df['Year'].astype(str) + df['CalendarWeek'].astype(str) + '1', format='%Y%W%w')
df['Month'] = df['Date'].dt.month
df = df.drop(columns=['Date'])

In [32]:
# existing code...
df.loc[df['Risikogruppe'] == 'no', 'Risikogruppe'] = 'hohem Risiko'
df.loc[df['Risikogruppe'] == 'yes', 'Risikogruppe'] = 'niedrigem Risiko'
df.loc[df['Altersgruppe'] == '0-29', 'Altersgruppe'] = 'unter 60 jährige'
df.loc[df['Altersgruppe'] == '30-59', 'Altersgruppe'] = 'unter 60 jährige'
df.loc[df['Altersgruppe'] == '60-199', 'Altersgruppe'] = 'über 60 jährige'

In [33]:
df

,Region,Altersgruppe,Inzidenz,Year,CalendarWeek,Time,Risikogruppe,Percentage,Inzidenz_cumsum,Percentage_cumsum,Month
2629,Baden-Württemberg,unter 60 jährige,0.00964,2024,1,2024-01-01,hohem Risiko,0.002757,0.002757,0.00964,1
2677,Baden-Württemberg,unter 60 jährige,0.01069,2024,2,2024-01-08,hohem Risiko,0.003057,0.005814,0.02033,1
2725,Baden-Württemberg,unter 60 jährige,0.02613,2024,3,2024-01-15,hohem Risiko,0.007473,0.013288,0.04646,1
2773,Baden-Württemberg,unter 60 jährige,0.04003,2024,4,2024-01-22,hohem Risiko,0.011449,0.024736,0.08649,1
2821,Baden-Württemberg,unter 60 jährige,0.04293,2024,5,2024-01-29,hohem Risiko,0.012278,0.037014,0.12942,1
...,...,...,...,...,...,...,...,...,...,...,...
2436,Thüringen,über 60 jährige,0.06115,2025,5,2025-01-27,niedrigem Risiko,0.043661,0.301694,0.15654,2
2484,Thüringen,über 60 jährige,0.04948,2025,6,2025-02-03,niedrigem Risiko,0.035329,0.337022,0.20602,2
2532,Thüringen,über 60 jährige,0.06434,2025,7,2025-02-10,niedrigem Risiko,0.045939,0.382961,0.27036,2
2580,Thüringen,über 60 jährige,0.05558,2025,8,2025-02-17,niedrigem Risiko,0.039684,0.422645,0.32594,2


In [34]:
dff = df.groupby(['Year', 'Month', 'Region', 'Risikogruppe', 'Altersgruppe']).sum().reset_index()

In [35]:
dff

,Year,Month,Region,Risikogruppe,Altersgruppe,Inzidenz,CalendarWeek,Time,Percentage,Inzidenz_cumsum,Percentage_cumsum
0,2024,1,Baden-Württemberg,hohem Risiko,unter 60 jährige,0.20597,30,2024-01-012024-01-082024-01-152024-01-222024-0...,0.058907,0.129790,0.45381
1,2024,1,Baden-Württemberg,hohem Risiko,über 60 jährige,0.10925,15,2024-01-012024-01-082024-01-152024-01-222024-0...,0.031245,0.068803,0.24057
2,2024,1,Baden-Württemberg,niedrigem Risiko,unter 60 jährige,0.20597,30,2024-01-012024-01-082024-01-152024-01-222024-0...,0.147063,0.324020,0.45381
3,2024,1,Baden-Württemberg,niedrigem Risiko,über 60 jährige,0.10925,15,2024-01-012024-01-082024-01-152024-01-222024-0...,0.078004,0.171767,0.24057
4,2024,1,Bayern,hohem Risiko,unter 60 jährige,0.36448,30,2024-01-012024-01-082024-01-152024-01-222024-0...,0.104241,0.221501,0.77448
...,...,...,...,...,...,...,...,...,...,...,...
945,2025,3,Schleswig-Holstein,niedrigem Risiko,über 60 jährige,0.04619,9,2025-02-24,0.032980,0.284807,0.27017
946,2025,3,Thüringen,hohem Risiko,unter 60 jährige,0.28964,18,2025-02-242025-02-24,0.082837,0.947209,1.46776
947,2025,3,Thüringen,hohem Risiko,über 60 jährige,0.05903,9,2025-02-24,0.016883,0.186177,0.38497
948,2025,3,Thüringen,niedrigem Risiko,unter 60 jährige,0.28964,18,2025-02-242025-02-24,0.206803,2.364711,1.46776


In [36]:
cache = Cache()

def send_row_to_model(row):
    previous_record = cache.get_from_cache(row['Region'], row['Year'], row['Month'] - 1, row['Risikogruppe'], row['Altersgruppe'])
    previous_rate = previous_record['InfectionRate'] if previous_record else 0.0926

    colname_fallzahlen = 'Percentage'
    prompt = generate_prompt(row['Region'], row['Year'], row['Month'], row[colname_fallzahlen], previous_rate, row['Risikogruppe'], row['Altersgruppe'])
    
    model_output = lm(prompt)
    parsed_value = parse_lm_output(model_output)
    print("Parsed value:", parsed_value, "  " ,"Previous Rate:", previous_rate)

    cache.add_to_cache(
        region=row['Region'],
        year=row['Year'],
        month=row['Month'],
        percentage=row[colname_fallzahlen],
        infection_rate=parsed_value,  # i.e. InfectionRate
        prompt=prompt,
        output=model_output,
        risikogruppe=row['Risikogruppe'],
        altersgruppe=row['Altersgruppe'],
        previous_infection_rate=previous_rate
    )
    
    return parsed_value

In [37]:
# dff = dff[dff['Region'] == 'Baden-Württemberg']
# dff = dff[dff['Month'].isin([7, 8])]
len(dff)

8

In [38]:
dff['predicted_value'] = dff.apply(lambda x: send_row_to_model(x), axis=1)

Parsed value: 0.005    Previous Rate: 0.0926
Parsed value: 0.0002    Previous Rate: 0.0926
Parsed value: 0.0758    Previous Rate: 0.0926
Parsed value: 0.0412    Previous Rate: 0.0926
Parsed value: 0.0015    Previous Rate: 0.005
Parsed value: 0.00015    Previous Rate: 0.0002
Parsed value: 0.0006    Previous Rate: 0.0758
Parsed value: 0.0123    Previous Rate: 0.0412


In [39]:
dff = dff[dff['Region'] == 'Baden-Württemberg']
dff.sort_values(by=['Year', "Month"], inplace=True)

In [40]:
dff

,Year,Month,Region,Risikogruppe,Altersgruppe,Inzidenz,CalendarWeek,Time,Percentage,Inzidenz_cumsum,Percentage_cumsum,predicted_value
382,2024,7,Baden-Württemberg,hohem Risiko,unter 60 jährige,0.00064,232,2024-07-012024-07-152024-07-292024-07-012024-0...,0.000183,0.478384,0.02056,0.00500
383,2024,7,Baden-Württemberg,hohem Risiko,über 60 jährige,0.00120,145,2024-07-012024-07-082024-07-152024-07-222024-0...,0.000343,0.365986,0.01447,0.00020
384,2024,7,Baden-Württemberg,niedrigem Risiko,unter 60 jährige,0.00064,232,2024-07-012024-07-152024-07-292024-07-012024-0...,0.000457,1.194286,0.02056,0.07580
385,2024,7,Baden-Württemberg,niedrigem Risiko,über 60 jährige,0.00120,145,2024-07-012024-07-082024-07-152024-07-222024-0...,0.000857,0.913684,0.01447,0.04120
444,2024,8,Baden-Württemberg,hohem Risiko,unter 60 jährige,0.00109,268,2024-08-052024-08-122024-08-192024-08-262024-0...,0.000312,0.509383,0.02614,0.00150
445,2024,8,Baden-Württemberg,hohem Risiko,über 60 jährige,0.00060,134,2024-08-052024-08-122024-08-192024-08-26,0.000172,0.293811,0.01515,0.00015
446,2024,8,Baden-Württemberg,niedrigem Risiko,unter 60 jährige,0.00109,268,2024-08-052024-08-122024-08-192024-08-262024-0...,0.000778,1.271677,0.02614,0.00060
447,2024,8,Baden-Württemberg,niedrigem Risiko,über 60 jährige,0.00060,134,2024-08-052024-08-122024-08-192024-08-26,0.000428,0.733499,0.01515,0.01230


In [41]:
dff['previous'] = dff.apply(lambda x: cache.get_from_cache(x['Region'], x['Year'], x['Month'], x['Risikogruppe'], x['Altersgruppe'])['PreviousInfectionRate'], axis=1)

In [42]:
dff

,Year,Month,Region,Risikogruppe,Altersgruppe,Inzidenz,CalendarWeek,Time,Percentage,Inzidenz_cumsum,Percentage_cumsum,predicted_value,previous
382,2024,7,Baden-Württemberg,hohem Risiko,unter 60 jährige,0.00064,232,2024-07-012024-07-152024-07-292024-07-012024-0...,0.000183,0.478384,0.02056,0.00500,0.0926
383,2024,7,Baden-Württemberg,hohem Risiko,über 60 jährige,0.00120,145,2024-07-012024-07-082024-07-152024-07-222024-0...,0.000343,0.365986,0.01447,0.00020,0.0926
384,2024,7,Baden-Württemberg,niedrigem Risiko,unter 60 jährige,0.00064,232,2024-07-012024-07-152024-07-292024-07-012024-0...,0.000457,1.194286,0.02056,0.07580,0.0926
385,2024,7,Baden-Württemberg,niedrigem Risiko,über 60 jährige,0.00120,145,2024-07-012024-07-082024-07-152024-07-222024-0...,0.000857,0.913684,0.01447,0.04120,0.0926
444,2024,8,Baden-Württemberg,hohem Risiko,unter 60 jährige,0.00109,268,2024-08-052024-08-122024-08-192024-08-262024-0...,0.000312,0.509383,0.02614,0.00150,0.0050
445,2024,8,Baden-Württemberg,hohem Risiko,über 60 jährige,0.00060,134,2024-08-052024-08-122024-08-192024-08-26,0.000172,0.293811,0.01515,0.00015,0.0002
446,2024,8,Baden-Württemberg,niedrigem Risiko,unter 60 jährige,0.00109,268,2024-08-052024-08-122024-08-192024-08-262024-0...,0.000778,1.271677,0.02614,0.00060,0.0758
447,2024,8,Baden-Württemberg,niedrigem Risiko,über 60 jährige,0.00060,134,2024-08-052024-08-122024-08-192024-08-26,0.000428,0.733499,0.01515,0.01230,0.0412
